# Work through the following Notebook and answer the listed questions

# Introduction/Motivation

In addition to working with numerical computations, it's possible to use python in symbolic work. The necessary library is called `sympy`. One of the nice features of `sympy` (especially in a notebook) is that it will format mathematical output to look more like math than like code.  To ask `sympy` to do this, call the `init_printing` function.

This tutorial is based around `sympy` version 1.3. Earlier versions may lack some functionality; later versions may be more capable or have changed apis.

In [ ]:
pip install sympy #Run to install sympy if not already installed

In [ ]:
pip install matplotlib #For plotting if not already installed

In [ ]:
# Imports for this tutorial
import sympy as sp

sp.init_printing()

sp.__version__

# Simplest use cases

The first hurdle you will need to overcome in symbolic work is to keep clear in your mind the distinction between mathematical *symbols* and python *variables*. We often use the word *variable* to refer to mathematical quantities, but in the context of `sympy` that is likely to lead to confusion. I'll try to be consistent in this tutorial to only use the word *variable* to refer to a reference to a python object, and I'll use the word *symbol* to refer to mathematical objects.

Unfortunately, we will have to use python variables to refer to `sympy` symbols; if you're not careful, you can get confused.  Before we can use a symbol, we have to declare it and assign it to a variable. We can do both in a single statement:

In [ ]:
x = sp.symbols('x')

You can instantiate several symbols at a time and assign them all to variables:

In [ ]:
a,b,c = sp.symbols('a b c')

Symbol names are separated by spaces or commas.  Greek letters are supported by name, as well:

In [ ]:
alpha, beta, gamma, lam = sp.symbols('alpha, beta, gamma, lambda')
alpha, beta, gamma, lam

However, note that `lambda` is a reserved word in python (it's used for anonymous functions) so if you have a $\lambda$ in your math you'll need to give the python variable a name other than `lambda`.

## Algebra

Okay, this has been boring so far. Let's do something a little more interesting! Let's try multiplying a couple of polynomials.

In [ ]:
poly1 = a*x**2 + b*x + c
poly2 = alpha*x**2 + beta*x + lam

poly1

In [ ]:
poly1*poly2

If we want to expand this, we need to ask for it explicitly.

In [ ]:
sp.expand(poly1*poly2)

Most methods can either be called from the module directly (as above), or they can be chained off of `sympy` expressions:

In [ ]:
(poly1**4).expand()

In the preceeding I used parentheses to make sure the call to `expand()` was applied to the whole expression rather than just the number 4.

We can also solve symbolically (factoring when possible):

In [ ]:
sp.solve(poly1, x)

The core of the syntax is `sp.solve(expr, symbol)`, where `expr` is the expression you're solving, and `symbol` is the `sympy` symbol for which you are solving.

The `solve` method can work on two different types of expressions: with or without a statement about equality. If your expression doesn't contain an equals sign, (as above), the right hand side of the expression is assumed to be zero.

### A side note about equality
The mathematical symbol `=` can mean one of several different things when we use it on paper. This kind of ambiguity doesn't work well for computers, so we have to introduce different python syntax for the different meanings of `=`. In order of least to most complicated syntax:

- **Assignment operator** in a statement like `a=4`, the `=` means "assign the value on the right to the variable on the left." If the thing on the left isn't a variable, python will throw an exception. Otherwise, it changes the value of the variable and returns nothing.

- **Test of equality** in a statement like `a==4`, the `==` means "True or False: the value of the thing on the left the same as the value of the thing on the right." 

- **A statement of mathematical equality** This is the new one. We use the `sympy` function `Eq` to create a mathematical expression that asserts equality: `Eq(a,4)`. This code means "the symbol (or collection of symbols) contained in the python variable `a` is mathematically equal to 4." it returns a `sympy` expression which makes that assertion.

In [ ]:
sp.Eq(poly1, poly2)

## Question 1: Explain what the command above does in your own words
 (Explanation here)

In [ ]:
sp.solve(sp.Eq(poly1, poly2), x)

Note that `solve` doesn't return a `sympy` expression; it returns a *python list* of expressions. Why is this important? Hint: `sympy` functions only work on `sympy` expressions.

A fourth order equation has four solutions, so `solve` returns four items in the list:

In [ ]:
sp.solve(poly1*poly2, x)

## Question 2: Recall from Worksheet 1 that we found $x$ for $3x^2 - 5x + 3 = 1$. Use sympy to solve the equation and compare your results below
Hint: Use sp.solve(). You will need to rearrange the equation below before using sp.solve

In [ ]:
#3*x**2-5*x+3=1

We can manage complex roots just fine.

In [ ]:
sp.solve(x**3 + c, x)

If our expression contains numbers in addition to symbols, those are handled (somewhat) gracefully, but the bias is always toward returning a symbolic result of some kind.

In [ ]:
sp.solve(x**3 - 7*x**2 - 2*x + 4, x)

If we want a numerical result, we have to ask for it with either `N` or `evalf`. The syntax is slightly different, but they give the same result. 


**Side note:**
Since `solve` returns a list of solutions, I have to apply the appropriate method to each element of the list. The pythonic way to do this is with a list comprehension.

In [ ]:
[sp.N(ans) for ans in sp.solve(x**3 - 7*x**2 - 2*x + 4, x)]

In [ ]:
[ans.evalf() for ans in sp.solve(x**3 - 7*x**2 - 2*x + 4, x)]

Okay, so there's something odd going on here. We're getting some really small imaginary parts in our solutions. Generally speaking, when a numerical calculation returns something that small (especially when it's added to something much larger) it's failing to correctly represent zero. If we plot this, can we see three real roots? 

(Hey, `sympy` can plot, too! this is a little more convenient that the plotting we've done with `matplotlib` in the past, though it does use `matplotlib` under the hood. As a reminder, the `%matplotlib inline` magic displays plots inline in the notebook.)

The syntax to plot a function of a single variable is `plot(expr, range)` where `expr` is a symbolic expression, and `range` is a 3-tuple of the form `(symbol, min, max)`.

In [ ]:
# so we can see the plot in the notebook
%matplotlib inline

sp.plot((x**3 - 7*x**2 - 2*x + 4), (x, -2.5, 7.5))

We have three real roots, so why the spurious tiny imaginary parts?  What's happening here is that `sympy` is finding the difference of two values which *should* exactly cancel, but due to finite precision, don't. This is a known problem, though, so it has a built-in resolution: the `chop` keyword.

In [ ]:
[ans.evalf(chop=True) for ans in sp.solve(x**3 - 7*x**2 - 2*x + 4, x)]

These answers line up with those shown on the plot, and no longer have the distracting almost-zero imaginary parts.

## Question 3: Based on the results above what are the solutions to $ x^3 - 7x^2-2x+4$?
(answer here)

If we want to evaluate the function at a single point, we have to do a substitution first, and then an evaluation. The `evalf` method has a keyword for this:

In [ ]:
cubic = x**3 - 7*x**2 - 2*x + 4

cubic.evalf(subs={x:1})

but we can use the `subs` method for more general substitutions.

In [ ]:
cubic.subs({x:a})

Both the `subs` method and the keyword to `evalf` take dictionaries as arguments, so that you can substitute multiple symbols at once.

In [ ]:
poly1.subs({a:3, b:-5, c:6})

In [ ]:
# Sympy doesn't plot functions with asymptotes elegantly
%matplotlib inline

sp.plot((x**3 - 7*x**2 - 2*x + 4)**(-1), (x, -2.5, 7.5))

sp.plot((x**3 - 7*x**2 - 2*x + 4)**(-1), (x, -2.5, 7.5),ylim=[-2,2])

## Simple calculus

As you might expect, we can compute derivatives and integrals with `sympy`, too.

In [ ]:
sp.diff(poly1, x)

In [ ]:
sp.integrate(poly1, x)

In [ ]:
sp.integrate(poly1, (x, 0, 5))

If you're working with functions that are more complicated than simple arithmetic compositions, be sure to use the `sympy` versions.

In [ ]:
func = sp.exp(-2*x**2)*sp.sin(3*sp.pi*x)

In [ ]:
sp.plot(func, (x, -2, 2))

## Question 4: Calculate at least one derivative **and** one integral of your choice below

In [ ]:
funcprime = sp.diff(func, x)
funcprime

If `sympy` can't figure out how to do an integral, it will return the integral without evaluating it. This is even true of integrals which could be calculated numerically; you have to use a different approach (about which more later) to deal with them.

In [ ]:
sp.integrate(func, x)

In [ ]:
sp.integrate(func, (x, 0,1))

One advantage `sympy` has for physics students is that it is very careful about convergence conditions we tend to ignore (or treat in a sloppy way). It's also aware of a whole host of special functions.

In [ ]:
sp.integrate(x**a*sp.exp(-x), (x, 0, sp.oo))

You can deal with some of these conditions by constraining your symbols (see the appropriate section below).

## Series (We can also look at series with sympy)

In [ ]:
sp.series(sp.sin(x),n=15)

## Question 5: Show the first 5 terms of arctangent below

# Additional information

In [ ]:
help(sp.plot)